In [ ]:
product_id = 1729617489563257366

In [ ]:
from Parallelizer import make_parallel
from common import  Common
import re
import pandas as pd
import time
from urllib.parse import unquote


In [ ]:
prices = "₫145.000 - ₫155.000"


In [ ]:
def convert_price(price):
    if price and type(price) == str:
        price_characters = ["K", "M", "B"]
        for i, character in enumerate(price_characters):
            if character in price or character.lower() in price:
                price = price.replace(character, "0" * (3 - i))
                price = price.replace(character.lower(), "0" * (3 - i))
                break
        if "," in price:
            price = price.replace(",", "")
            price = price[:-1]
    return price


def parse_input(path_string):
    output = []
    for i in path_string.split('.'):
        if '[' in i:
            output += map(lambda x: int(x) if x.isdigit() else x, i.replace(']', '').split('['))
        else:
            output.append(i)
    return output

In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep
count = 0

# @make_parallel
def request_tiktok(product_id):
    payload = {}
    headers = {
        'authority': 'oec-api.tiktokv.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-GB,en;q=0.9,en-US;q=0.8',
        'cache-control': 'max-age=0',
        'sec-ch-ua': '"Microsoft Edge";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.47'
    }
    try_time = 0
    json_obj = None
    while not json_obj:
        
        url = f"https://oec-api.tiktokv.com/view/product/{product_id}"
        response = requests.request("GET", url, headers=headers, data=payload)
        try_time += 1
        print(f"{try_time}: {url}")
        try:
            soup = BeautifulSoup(response.text, 'html.parser')

            # get content of script id = RENDER_DATA
            script = soup.find('script', id='RENDER_DATA')
            # get content of script
            script_content = script.contents[0]
            # get data from script
            # print(response.text)
            json_obj = unquote(script_content)
            # print(json_obj)
        except Exception as e:
            print(e)
            json_obj = None
            print(f"fail: {product_id} {response.text}")
            sleep(2)
    return json_obj

In [ ]:
a = request_tiktok(1729592140551784572)

In [ ]:
# products_info = []
for product_id in product_ids:
    products_info.append(request_tiktok(product_id))
    print("number product ",len(products_info))

In [ ]:
import requests
import json
#  /v1/shop/products/{user_id}/{product_id}
@make_parallel
def get_product_info(product_id):
    print(f"Getting product info for {product_id}")
    url = f"https://tokapi-mobile-version.p.rapidapi.com/v1/shop/product/{str(product_id)}/reviews"
    # url = f"https://tokapi-mobile-version.p.rapidapi.com//v1/shop/products/7494645360231549052/{product_id}"

    querystring = {"region":"VN"}

    headers = {
        "X-RapidAPI-Key": "13e1f4409dmshffaa6951f986e56p189473jsneb9f284bacbc",
        "X-RapidAPI-Host": "tokapi-mobile-version.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    json_object = response.json()
    if not json_object.get('data'):
        print(f"Failed to get product info for {product_id}")
    else:
        print(f"Got product info for {product_id}")
    return json_object

In [ ]:
product_infos = list(map(json.loads, products_info))

In [ ]:
import json
with open("tiktok_product_info.json", "w", encoding="utf8") as f:
    json.dump(product_infos, f, ensure_ascii=False, indent=4)

In [ ]:
test = get_product_info([1729639375712192636])
print(json.dumps(test, ensure_ascii=False))

In [ ]:
product_ids =[1729592140551784572,1729449511894157436,1729658387123832956,1729431748019325052,1729626317079218300,1729572679537690748,1729440105127118972,1729451561887434876,1729437324925110396,1729630659136817276,1729583724521031804,1729626846098131068,1729509471228692604,1729605709738772604,1729565467366295676,1729620755435915388,1729446849650919548,1729512000393873532,1729535201399441532,1729599337592555644,1729553523643287676,1729479614583113852,1729501770531702908,1729572678106318972,1729475429215078524,1729601206794225788,1729501769869265020,1729691242946267260,1729431747192129660,1729439080686717052,1729639120843212924,1729626623443372156,1729535202367146108,1729432692253296764,1729624758677244028,1729557070413596796,1729431748429908092,1729431747187804284,1729502787769043068,1729431747934980220,1729599963628472444,1729611038363125884,1729615854231849084,1729501767912687740,1729431747592095868,1729718297431214204,1729431747540977788,1729435534075005052,1729557007359707260,1729626822072633468,1729447849067710588,1729611649591249020,1729432440521066620,1729552584743684220,1729643035789002876,1729618510169868412,1729647577519327356,1729431747650226300,1729600513778354300,1729549339019282556,1729431748112844924,1729658651792541820,1729445014411905148,1729572677865736316,1729535201877264508,1729687741809723516,1729501757725771900,1729455028301367420,1729699305468627068,1729431747894872188,1729523504395159676,1729511971905505404,1729535201822476412,1729602688941459580,1729511944412301436,1729472350990665852,1729643051475699836,1729647569733519484,1729639267271411836,1729434609401759868,1729557711216806012,1729603411521144956,1729589595173587068,1729570370282162300,1729431747191146620,1729549324909840508,1729635570144413820,1729531472213084284,1729463219986991228,1729709317573937276,1729659547713177724,1729572019212224636,1729581937004021884,1729511937864992892,1729653547890018428,1729447527098583164,1729431747458467964,1729611651602811004,1729624461755254908,1729431747994683516,1729435535153531004,1729431747968338044,1729557713038116988,1729620001208699004,1729535203359623292,1729439354518014076,1729620757844625532,1729431747649505404,1729646638461585532,1729431747674933372,1729575664730081404,1729447850460678268,1729570744126441596,1729504547363260540,1729634541222922364,1729452374104115324,1729602620299774076,1729572178500028540,1729589595896776828,1729601203808274556,1729562361650120828,1729602664083589244,1729431747478063228,1729502770443749500,1729540836358391932,1729506326880422012,1729501769648474236,1729701222487984252,1729639273496086652,1729575735088416892,1729545679386413180,1729647535944599676,1729596149897660540,1729545672755021948,1729431748010150012,1729502805186676860,1729620003391375484,1729431748013164668,1729535201162528892,1729435532721752188,1729501770021439612,1729641198040156284,1729619837773711484,1729602694270257276,1729456028398291068,1729687764777928828,1729431747539929212,1729559831898982524,1729570738555357308,1729431747661301884,1729599337628665980,1729548315344275580,1729446843070187644,1729502771214780540,1729435535055161468,1729588178107467900,1729570351953250428,1729565477318854780,1729431747540387964,1729687820518197372,1729431747679324284,1729431747461351548,1729441583108556924,1729431747747547260,1729431747493267580,1729549333805631612,1729653561459181692,1729479602638588028,1729593759883036796,1729431747324512380,1729449100165744764,1729634917517068412,1729709321654208636,1729548303880980604,1729604664677730428,1729579439531526268,1729501768546289788,1729583724803164284,1729572677636491388,1729572652480628860,1729435528983840892,1729446850110982268,1729658521095604348,1729639377495033980,1729439065352472700,1729451551727847548,1729448661617051772,1729603429725014140,1729529088867469436,1729565494241626236]

In [ ]:
product_infos = get_product_info(product_ids)

In [ ]:
@make_parallel
def transform(response, results = []):
    
    
    try:
        product_base = response['data']['products'][0]['product_base']
        mapping_product_base = {
            "real_price": ["price", "real_price"],
            "original_price": ["price", "original_price"],
            "images": ["images"],
            "sold_count": ["sold_count"],
            "title": ["title"]
        }
        transformed_product_base = Common.mapping_data(
            product_base, mapping_product_base)
        product_id = response['data']['products'][0]['product_id']
        
        print(product_id)
        name = transformed_product_base['title']
        price_pattern = re.compile(r'(\d+\.?\d*)')
        prices = price_pattern.findall(transformed_product_base['real_price'])
        if len(prices) == 2:
            min_price = int(prices[0].replace('.', ''))
            max_price = int(prices[1].replace('.', ''))
        else:
            min_price = None
            max_price = int(prices[0].replace('.', ''))
            
        original_price = price_pattern.findall(transformed_product_base['original_price'])
        if len(original_price) == 2:
            min_original_price = int(original_price[0].replace('.', ''))
            max_original_price = int(original_price[1].replace('.', ''))
        else:
            min_original_price = None
            if len(original_price) == 0:
                max_original_price = None
            else:
                max_original_price = int(original_price[0].replace('.', ''))

        sold_count = transformed_product_base['sold_count']
        image_1 = f'=image("{transformed_product_base["images"][0]["thumb_url_list"][0]}")'
        image_2 = f'=image("{transformed_product_base["images"][1]["thumb_url_list"][0]}")' if len(transformed_product_base['images']) > 1 else None
        image_3 = f'=image("{transformed_product_base["images"][2]["thumb_url_list"][0]}")' if len(transformed_product_base['images']) > 2 else None
        
        results.append({
            "product_id": product_id,
            "link": f"https://oec-api.tiktokv.com/view/product/{product_id}",
            "name": name,
            "min_price": min_price,
            "max_price": max_price,
            "min_original_price": min_original_price,
            "max_original_price": max_original_price,
            "sold_count": sold_count,
            "image_1": image_1,
            "image_2": image_2,
            "image_3": image_3
        })
    except Exception as e:
        print(e)
        print(f"failed {response}")
        results.append({
                "product_id": "failed",
                "link": "failed",
                "min_price": "failed",
                "max_price": "failed",
                "min_original_price": "failed",
                "max_original_price": "failed",
                "sold_count": "failed",
                "image_1": "failed",
                "image_2": "failed",
                "image_3": "failed",
            })

In [ ]:
results = []
transform(product_infos, results)

In [ ]:
df = pd.DataFrame(results)
df.to_csv('product_infos.csv', index=False)

In [ ]:
driver.quit()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize the WebDriver (e.g., Chrome)
driver = webdriver.Chrome()

# Define the URL you want to visit
url = "https://shopee.vn/becungstore08"

# Open the URL
driver.get(url)

# Wait for the page to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#main")))

# Find the elements using the specified CSS selector
product_elements = driver.find_elements(By.CSS_SELECTOR, "#main > div > div:nth-child(3) > div > div > div > div.shop-page > div > div.container > div.shop-page__all-products-section > div.shop-page_product-list > div > div.shop-search-result-view")

# Loop through the elements and print their text or other relevant information
for product_element in product_elements:
    print(product_element.text)

# Close the WebDriver
# driver.quit()


In [ ]:
ID_PATTERN = r'-i.(\d+).(\d+)'

In [ ]:
product_crawled = []

In [ ]:
len(product_crawled)

In [ ]:
while True:
    try:
        button_element = driver.find_element(
            By.XPATH, "//button[contains(@class, 'shopee-mini-page-controller__next-btn') and not(contains(@class, 'shopee-button-outline--disabled'))]")
    except:
        button_element = None
    # Check if the button is enabled (does not have 'shopee-button-outline--disabled' class)

    # Find elements with the class "shop-search-result-view"
    product_elements = driver.find_elements(
        By.CSS_SELECTOR, ".shop-search-result-view")

    # Loop through the elements and get the href attribute
    for product_element in product_elements:
        a_tags = product_element.find_elements(By.TAG_NAME, "a")
        for a_tag in a_tags:
            href = a_tag.get_attribute("href")
            # print(href)
            id_match = re.search(ID_PATTERN, href)
            sold_out = None
            image = None
            try:
                sold_out = a_tag.find_element(By.CSS_SELECTOR, ".sPnnFI").text
                image = a_tag.find_element(By.TAG_NAME, "img").get_attribute("src")
                print(sold_out)
            except Exception as e:
                print(f"failed {e}")
                pass
            print(id_match[1], id_match[2])
            product_crawled.append({
                "link": href,
                "sold_out": sold_out or "0",
                "product_id": id_match[2],
                "image": f'=image("{image}")'
            })
    if not button_element:
        while True:
            try:
                div_element = driver.find_element(
                    By.CLASS_NAME, 'shop-sold-out-see-more')
            except:
                div_element = None
                print("No more sold out")

            # Find the button inside the div element
            try:
                button_element = div_element.find_element(
                    By.TAG_NAME, 'button')
            except:
                button_element = None
                print("No more button")

            if button_element:
                time.sleep(1)
                button_element.click()

            else:
                product_elements = driver.find_elements(
                    By.CSS_SELECTOR, ".shop-sold-out-items-view")

                # Loop through the elements and get the href attribute
                for product_element in product_elements:
                    a_tags = product_element.find_elements(By.TAG_NAME, "a")
                    for a_tag in a_tags:
                        href = a_tag.get_attribute("href")
                        print(href)
                        id_match = re.search(ID_PATTERN, href)
                        sold_out = None
                        image = None
                        try:
                            sold_out = a_tag.find_element(By.CSS_SELECTOR, ".sPnnFI").text
                            image = a_tag.find_element(By.TAG_NAME, "img").get_attribute("src")
                            print(sold_out)
                        except:
                            pass
                        print(id_match[1], id_match[2])
                        product_crawled.append({
                            "link": href,
                            "sold_out": sold_out or "0",
                            "product_id": id_match[2],
                            "image": f'=image("{image}")'
                        })

            if not div_element:
                break
        break
    time.sleep(1)
    button_element.click()

# Close the WebDriver



In [ ]:
len(product_crawled)

In [ ]:
while True:
    try:
        div_element = driver.find_element(
            By.CLASS_NAME, 'shop-sold-out-see-more')
    except:
        div_element = None

    # Find the button inside the div element
    try:
        button_element = div_element.find_element(
            By.TAG_NAME, 'button')
    except:
        button_element = None

    if button_element:
        time.sleep(1)
        button_element.click()

    else:
        product_elements = driver.find_elements(
            By.CSS_SELECTOR, ".shop-sold-out-items-view")

        # Loop through the elements and get the href attribute
        for product_element in product_elements:
            a_tags = product_element.find_elements(By.TAG_NAME, "a")
            for a_tag in a_tags:
                href = a_tag.get_attribute("href")
                print(href)
                id_match = re.search(ID_PATTERN, href)
                sold_out = None
                image = None
                try:
                    sold_out = a_tag.find_element(By.CSS_SELECTOR, ".sPnnFI").text
                    image = a_tag.find_element(By.TAG_NAME, "img").get_attribute("src")
                    print(sold_out)
                except:
                    pass
                print(id_match[1], id_match[2])
                product_crawled.append({
                    "link": href,
                    "sold_out": sold_out,
                    "product_id": id_match[2],
                    "image": f'=image("{image}")'
                })
    if not div_element:
        break


In [ ]:
len(product_crawled)

In [ ]:
len(product_crawled), len(product_details), len(product_details_failed), len(product_details_processed)

In [ ]:
product_details_failed

In [ ]:
product_details = []
product_details_failed = []
product_details_processed = set()
product_details_failed_set = set()

In [ ]:
for item in product_crawled:
    if len(str(item['product_id'])) < 2:
        link = unquote(item['link'])
        link = link[:link.find("?")]
        print(item['link'])
        id_match = re.search(ID_PATTERN, link)
        item['product_id'] = id_match[2]
        item['link'] = link
        # break

In [ ]:
data = {
            "item_id" : int(19158046466),
            "shop_id" : 66584257,
            "quantity" : 1,
        }

item_info = requests.post(url= url, json = data).json()
print(j)

In [ ]:
import requests

@make_parallel
def get_product_details(item):
    try:
        product_id = item['product_id']
        if product_id in product_details_processed:
            print(f"Product {product_id} already processed")
            return
        # product_id = 18374615395
        sold_out = int(convert_price(item['sold_out']).split(" ")[-1]) if item['sold_out'] else None
        image = item['image']
        link = unquote(item['link'])
        link = link[:link.find("?")]
        url = "https://mall.shopee.vn/api/v4/pdp/cart_panel/get"
        print(link)
        data = {
            "item_id" : int(product_id),
            "shop_id" : 308839485,
            "quantity" : 1,
        }

        item_info = requests.post(url= url, json = data).json()
        if not image or "None" in image:
            image = item_info['data']['item']['image']
            image = f'=image("https://down-vn.img.susercontent.com/file/{image}")'
        time.sleep(.5)
        print(item_info)
        # break        
        min_price = max_price = min_original_price = max_original_price = None
        
        for model in item_info['data']['item']['models']:
            price = int(convert_price(model['price']['single_value'])) /100000 if model['price'] else None
            original_price = int(convert_price(model['price_before_discount']['single_value'])) / 100000 if model['price_before_discount']['single_value'] >0 else None
        
            if not max_price:
                max_price = price
            elif price > max_price:
                min_price = max_price
                max_price = price
                
            try:
                if not max_original_price:
                    max_original_price = original_price
                elif original_price > max_original_price:
                    min_original_price = max_original_price
                    max_original_price = original_price
            except:
                pass
                
        name = item_info['data']['item']['tracking']['name']
        stock = int(item_info['data']['item']['stock'])
                
        product_details.append({
            "product_id": product_id,
            "link": link,
            "name": name,
            "min_price": int(min_price) if min_price else None,
            "max_price": int(max_price) if max_price else None,
            "min_original_price": int(min_original_price) if min_original_price else None,
            "max_original_price": int(max_original_price) if max_original_price else None,
            "sold_out": sold_out or "0",
            "image_1": image,
            "referrer_price": max_original_price or max_price,
            "out_stock": stock == 0,
        })
        product_details_processed.add(product_id)

        print(sold_out)
        print(json.dumps(item_info, ensure_ascii=False))
        print(len(product_details))
    except Exception as e:
        print(f"Failed to get product details for {product_id}")
        print(e)
        print(item)
        product_details_failed_set.add(product_id)
        product_details_failed.append(item)

In [ ]:
get_product_details(product_crawled)

In [ ]:
import requests

for item in product_crawled:
    try:
        product_id = item['product_id']
        if product_id in product_details_processed:
            print(f"Product {product_id} already processed")
            continue
        # product_id = 18374615395
        sold_out = int(convert_price(item['sold_out']).split(" ")[-1]) if item['sold_out'] else None
        image = item['image']
        if not image:
            image = None
        link = unquote(item['link'])
        link = link[:link.find("?")]
        url = "https://mall.shopee.vn/api/v4/pdp/cart_panel/get"
        print(link)
        data = {
            "item_id" : int(product_id),
            "shop_id" : 66584257,
            "quantity" : 1,
        }

        item_info = requests.post(url= url, json = data).json()
        time.sleep(.5)
        print(item_info)
        # break        
        min_price = max_price = min_original_price = max_original_price = None
        
        for model in item_info['data']['item']['models']:
            price = int(convert_price(model['price']['single_value'])) /100000 if model['price'] else None
            original_price = int(convert_price(model['price_before_discount']['single_value'])) / 100000 if model['price_before_discount']['single_value'] >0 else None
        
            if not max_price:
                max_price = price
            elif price > max_price:
                min_price = max_price
                max_price = price
                
            try:
                if not max_original_price:
                    max_original_price = original_price
                elif original_price > max_original_price:
                    min_original_price = max_original_price
                    max_original_price = original_price
            except:
                pass
                
        name = item_info['data']['item']['tracking']['name']
        stock = int(item_info['data']['item']['stock'])
                
        product_details.append({
            "product_id": product_id,
            "link": link,
            "name": name,
            "min_price": int(min_price) if min_price else None,
            "max_price": int(max_price) if max_price else None,
            "min_original_price": int(min_original_price) if min_original_price else None,
            "max_original_price": int(max_original_price) if max_original_price else None,
            "sold_out": sold_out,
            "image_1": f'=image("{image}")',
            "referrer_price": max_original_price or max_price,
            "out_stock": stock == 0,
        })
        product_details_processed.add(product_id)

        print(sold_out)
        print(json.dumps(item_info, ensure_ascii=False))
        print(len(product_details))
    except Exception as e:
        print(f"Failed to get product details for {product_id}")
        print(e)
        print(item)
        product_details_failed_set.add(product_id)
        product_details_failed.append(item)

In [ ]:
import requests

for item in product_crawled:
    try:
        product_id = item['product_id']
        if product_id in product_details_processed:
            print(f"Product {product_id} already processed")
            continue
        # product_id = 18374615395
        sold_out = int(item['sold_out'].split(" ")[-1]) if item['sold_out'] else None
        link = unquote(item['link'])
        link = link[:link.find("?")]
        url = "https://mall.shopee.vn/api/v4/pdp/cart_panel/get"
        print(link)
        driver.get(link)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "product-briefing")))
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "img")))
        
        data = {
            "item_id" : int(product_id),
            "shop_id" : 64749808,
            "quantity" : 1,
        }

        item_info = requests.post(url= url, json = data).json()
        time.sleep(1.5)
        
        min_price = max_price = min_original_price = max_original_price = None
        
        for model in item_info['data']['item']['models']:
            price = int(model['price']['single_value']) /100000 if model['price'] else None
            original_price = int(model['price_before_discount']['single_value']) / 10000 if model['price_before_discount']['single_value'] else None
        
            if not max_price:
                max_price = price
            elif price > max_price:
                min_price = max_price
                max_price = price
                
            try:
                if not max_original_price:
                    max_original_price = original_price
                elif original_price > max_original_price:
                    min_original_price = max_original_price
                    max_original_price = original_price
            except:
                pass
                
        name = item_info['data']['item']['tracking']['name']
        stock = int(item_info['data']['item']['stock'])
                
        product_container = driver.find_element(By.CLASS_NAME, "product-briefing")
        images = product_container.find_elements(By.TAG_NAME, "img")
        image_1 = image_2 = image_3 = image_4 = image_5 = None
        for image in images[1:]:
            src = image.get_attribute("src")
            if "d67691c89049328180839b0df735788d" in src :
                continue
            if not image_1:
                image_1 = src
            elif not image_2:
                image_2 = src
            elif not image_3:
                image_3 = src
            elif not image_4:
                image_4 = src
            elif not image_5:
                image_5 = src
            else:
                break

        product_details.append({
            "product_id": product_id,
            "link": link,
            "name": name,
            "min_price": int(min_price) if min_price else None,
            "max_price": int(max_price) if max_price else None,
            "min_original_price": int(min_original_price) if min_original_price else None,
            "max_original_price": int(max_original_price) if max_original_price else None,
            "sold_out": sold_out,
            "image_1": f'=image("{image_2}")',
            "image_2": f'=image("{image_3}")',
            "image_3": f'=image("{image_4}")',
            "referrer_price": max_original_price or max_price,
            "out_stock": stock == 0,
        })
        product_details_processed.add(product_id)

        print(sold_out)
        print(json.dumps(item_info, ensure_ascii=False))
        print(len(product_details))
    except Exception as e:
        print(f"Failed to get product details for {product_id}")
        print(item)
        product_details_failed_set.add(product_id)
        product_details_failed.append(item)

In [ ]:
df = pd.DataFrame(product_details)
df.to_csv('product_details_shopee.csv', index=False)

In [ ]:
print(json.dumps(item_info['data'], ensure_ascii=False))

In [ ]:
product_crawled[0]

In [ ]:
image_1, image_2, image_3, image_4, image_5

In [ ]:
import json
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}
driver = webdriver.Chrome(desired_capabilities=caps)
driver.get('https://stackoverflow.com/questions/52633697/selenium-python-how-to-capture-network-traffics-response')

def process_browser_log_entry(entry):
    response = json.loads(entry['message'])['message']
    return response

browser_log = driver.get_log('performance') 
events = [process_browser_log_entry(entry) for entry in browser_log]
events = [event for event in events if 'Network.response' in event['method']]


In [ ]:
from selenium import webdriver
from seleniumwire import webdriver as wiredwebdriver
# Create a webdriver instance with seleniumwire
options = webdriver.ChromeOptions()
driver = wiredwebdriver.Chrome(options=options)


In [ ]:
from seleniumwire import webdriver  # Import from seleniumwire

# Create a new instance of the Chrome driver
driver = webdriver.Chrome()

# Go to the Google home page
driver.get('https://shopee.vn/atino.vn#product_list')

# Access requests via the `requests` attribute
for request in driver.requests:
    if request.response:
        if "shop_page_category_tab_main" in request.url:
            print(
                request.url,
                request.response.status_code,
                request.response.headers['Content-Type']
            )
        # print(
        #     request.url,
        #     request.response.status_code,
        #     request.response.headers['Content-Type']
        # )

In [ ]:
from selenium_stealth import stealth


In [ ]:
stealth(driver,
      languages=["en-US", "en"],
      vendor="Google Inc.",
      platform="Win32",
      webgl_vendor="Intel Inc.",
      renderer="Intel Iris OpenGL Engine",
      fix_hairline=True,
  )

In [ ]:
driver.get('https://shopee.vn/atino.vn#product_list')


In [ ]:
for request in driver.requests:
    if request.response:
        if "shop_page_category_tab_main" in request.url:
            print(
                request.url,
                request.response.status_code,
                request.response.headers['Content-Type']
            )

In [ ]:
from selenium import webdriver
from seleniumwire import webdriver as wiredwebdriver
# Create a webdriver instance with seleniumwire
options = webdriver.ChromeOptions()
options.add_argument( '--disable-blink-features=AutomationControlled' )
driver = wiredwebdriver.Chrome(options=options)
